In [ ]:
!pip install -U imbalanced-learn

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter
from scipy.special import softmax
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Exploratory Analysis

In [ ]:
data = pd.read_csv("/kaggle/input/website-phishing-data-set/Website Phishing.csv")
feats = data.columns.tolist()[:-1]
num_feats = len(feats)
y = data["Result"].tolist()
X = data.drop(columns=["Result"]).to_numpy()
classes = set(y)
print("Features: ", feats)
print("Classes: ", classes)

In [ ]:
feats = data.columns.tolist()
num_exem = len(data[feats[0]])
num_missing_values = 0
for feat in feats:
    num_missing_values += abs(len(data[feat]) - num_exem)
    
print("Number of missing values: ",num_missing_values)  

In [ ]:
for label in classes:
    print("Number of exemples of class {}: {}".format(label, len(data[data["Result"]==label])))

Apesar de não existirem dados faltando, o dataset está claramente desbalanceado. Isso pode afetar consideramente os resultados, porquê ainda que a avalições de acurácia sejam boas, alguns classes podem apresentar um nível de erro maior do que as demais principalmente aquelas que não possuem muitos exemplos , bem como, podemos enfretar casos de overfitting nas que tem muito. Considerando isso, uma boa tentativa de melhorar o desempenho é balancear o data-set. Usando *RandomOverSampler*, podemos balancear todas as classes, assim, as classes com menos exemplos teram a quantidade aumentada.

In [ ]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_balanced, y_balanced = ros.fit_resample(X, y)
print("Nova quantidade de exemplos: ", Counter(y_balanced))
y_balanced = np.array(y_balanced)

## Plot Information


Considerando que temos features que apresentam valores negativos, podemos utilizar a função softmax para garantir que todos os valores sejam positivos. Isso é importante para cálculo da média que poderia apresentar um valor negativo, o que é indesejado. Sendo assim, Isso garante a proporção dos dados e, portanto, o significado atribuido a eles.

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scl_X_balanced = scaler.fit_transform(X_balanced)
scl_X = scaler.fit_transform(X_balanced)

In [ ]:
def get_percentil(X):
    percent_array=np.zeros((num_feats, 100))

    # Calculates the percentiles for each feature
    for f in range(num_feats):
        for i in range(1,101):
            percent_array[f][i-1] = np.percentile(X[:,f], i)
 
    return percent_array

In [ ]:

def plot_info(feats_mean, feats_std, percent_array):
    fig, axis = plt.subplots(1,2,figsize=(16,5))

    plt.xlabel("Features")

    axis[0].hist(feats_mean) 
    axis[1].hist(feats_std)
    
    axis[0].legend(["Mean"])
    axis[1].legend(["Stantard Desviation"])
   
    plt.xlabel("Percentiles")
    fold1 = percent_array[:3]
    fold2 = percent_array[3:6]
    fold3 = percent_array[6:9]
    for fold in [fold1,fold2,fold2]:
        fig1, axis1 = plt.subplots(1,3,figsize=(25,10))
        for i, percent in enumerate(fold):
            axis1[i].plot(range(100), percent)
    fig1.subplots_adjust(wspace=0.5)

In [ ]:
feats_mean, feats_std = feats_mean = np.mean(X, axis=0), np.std(X, axis=0)
percent_array = get_percentil(X)
plot_info(feats_mean, feats_std, percent_array)

Como pode ser visto, há uma distribuição heterogênea das features o que pode fazer com que uma se sopreponha as demais por apresentar um desvio padrão acima da média. Além disso, o percentile mostra que os dados dos últimos valores (próximos a 100) tende a ter regiões um pouco distintas de concentração dos dados. Isso significa que o classificador terá, de alguma forma, possiveis regiões de classificação mais comuns, sendo mais díficil classificar um dado fora dessa tendência. 

In [ ]:
feats_mean, feats_std = feats_mean = np.mean(scl_X_balanced, axis=0), np.std(scl_X_balanced, axis=0)
percent_array = get_percentil(scl_X_balanced)
plot_info(feats_mean, feats_std, percent_array)


 Os resultados para o data-set balanceado apresentam um equilíbrio maior em relação a média de cada features o que consequentemente ajudará no processo de classificação. Além disso, a diminuisão do desvisão padrão criou um distribuição homogenea das features no data-set, isso faça com que as diferentes características dos dados sejam igualmente relevantes evitando que um sopreponha a outra. Isso permite que classificadores lineares como SVM, por exemplo, obtenha resultados consideravelmente melhores.

In [ ]:
from sklearn.cluster import KMeans
fig = plt.figure(1, figsize=(4, 3))

kmeans = KMeans(n_clusters=3)
kmeans.fit(scl_X_balanced[:100])
y_pred = kmeans.predict(scl_X_balanced[100:200])

plt.scatter(range(len(y_pred)), y_pred)

Como pode ser visto os dados podem ser agrupados e portanto separados por um classificador

# Classification



In [ ]:
classes = ['class '+str(c) for c in classes] # tornando as classes strings

# SVM -> Support Vector Machine 

In [ ]:
from sklearn import svm
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, shuffle=True)
svm_clf = svm.SVC()

svm_clf.fit(X_train,y_train)
preds = svm_clf.predict(X_test)

print("Training Count: ", Counter(y_train))
print("Testing Count: ", Counter(y_test))
print(classification_report(y_test, preds, target_names=classes))

## Análise Preliminar

Como pode ser visto, os resultados que utilizam os dados originais, apesar de terem um resultado considerável, apresentam um score totalmente desbalanceado em relação as classes. A classe 1, por exemplo, apresenta um recall de 0.16, enquanto a classe 0 tem um score de 0.92 representando uma diferença de 75%! Isso mostra que os dados do nosso classificador precisam ser balanceados e escalonados. Como vemos a seguir, quando isso acontece, obtemos resultados significativamente melhores. Isso ocorre tanto em relação as métricas obtidas em cada classe, como também, em relação a acurácia e as médias de recall, f1-score e precision.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(scl_X_balanced, y_balanced,test_size=0.2, shuffle=True)
print("Training Count: ", Counter(y_train))
print("Testing Count: ", Counter(y_test))

In [ ]:
#Classification using SVM -> Support Vector Machine 
# spliting training and validation

svm_clf = svm.SVC()

svm_clf.fit(X_train,y_train)
preds = svm_clf.predict(X_test)

svm_result = classification_report(y_test, preds, target_names=classes, output_dict=True)
print(classification_report(y_test, preds, target_names=classes))

## Análise 

Esse se apresenta como um dos melhores classificadores testados. Apesar de não termos um quantidade grande de dimenssões nos nossos dados (9 apenas), o classificador consegui gerar bem um hiperplano capaz de separar os dados. Além disso, o desempenho do classificador melhora consideravelmente quando regularizamos os dados, o que de fato é o esperado para esse tipo de classificador.
Como pode ser visto, ele apresenta uma acurácia de 0.92, o que um bom resultado. Além disso, a classificação de todas as classes obtem resultados consideravelmente bons igualmente. Considerando a precisão e recall, podemos afirmar que o classificador tem bons níveis de verdadeiro positivo como de verdadeiro negativo.

# Random Forest


In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf_rf = RandomForestClassifier()
clf_rf.fit(X_train, y_train)
preds = clf_rf.predict(X_test)
rf_result = classification_report(y_test, preds, target_names=classes, output_dict=True)
print(classification_report(y_test, preds, target_names=classes))

# Análise

O random forest assim como o Adabooster (próximo classificador), apresentam um abordagem distinta. Ao invés de utilizarem apenas um algoritmo para gerar as previsões, eles usam a combinação de vários classificadores para chegar a um predição mais precisa. De fato, é o que se apresenta nesse caso ontem a acurácia supera a do algoritmo SVM. Assim como o primeiro classificador (SVM), há um ótima distribuição de scores entre as classes, evitando que uma classe seja mais bem classificado do que as demais. Contudo, existe uma clara tendência de over-fitting, principalmente se analisarmos a class 1 onde os exemplos positivos tem um score de 1.0. Isso faz com que a árvore de decisão seja, também, um ótimo candidato.

# AdaBoostClassifier

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

clf_ada = AdaBoostClassifier(n_estimators=100)
y_pred = clf_ada.fit(X_train, y_train).predict(X_test)
ada_result = classification_report(y_test, y_pred, target_names=classes, output_dict=True)
print(classification_report(y_test, y_pred, target_names=classes))

## Análise

Apesar de ser um classificador baseado em diversos estimadores (nesse caso 100) o Adabooster não apresentam um resultado tão promissor quantos os demais. Ainda que mantenha uma boa distribuição dos acertos ao longo de todas as classes, o algoritimo, na média, apresenta um resultado inferior. Um dos possíveis motivos é o não melhor ajuste do meta-estimator aos dados, deixando muitos exemplos dífices para os demais estimators. 

In [ ]:
print(rf_result.keys())
metrics = ['accuracy']

results = []


for metric in metrics:
    score = []
    for i, r in enumerate([svm_result, rf_result, ada_result]):
        print(r[metric])
        score.append(r[metric]) 
    
    results.append(score)
    
for j,result in enumerate(results): 
    fig, axis = plt.subplots(1,1,figsize=(16,5))
    fig.suptitle('Resultado de Classificação Metric: {}'.format(metrics[j]), fontsize=16)
    for i, res in enumerate(results):
        print(res)
        axis.bar([1,2,3],res,
                    tick_label=['svm_result', 'rf_result', 'ada_result'], color=['green', 'blue', 'yellow'])

In [ ]:
# É preciso rolar o output para visualizar todas as métricas
metrics = ['f1-score', 'recall', 'precision']
metric_avg = ['macro avg', 'weighted avg']
for metric in metrics:
    for label in metric_avg:
        score = []
        for i, result in enumerate([svm_result, rf_result, ada_result]):
            result_f1 = result[label][metric]
            score.append(result_f1) 
        results.append(score)
    for j,label in enumerate(metric_avg): 
        fig, axis = plt.subplots(1,1,figsize=(5,3))
        fig.suptitle('Resultado de Classificação {} Metric: {}'.format(metric_avg[j], metric), fontsize=16)
        for i, result in enumerate(results):
            axis.bar([1,2,3],result,
                        tick_label=['svm_result', 'rf_result', 'ada_result'], color=['green', 'blue', 'yellow'])        

# Conclusão

Em resumo, os classificadores apresentam um bom desempenho no geral com destalhe para a SVM e o RandomForest. 
No requisito acuráicia o random forest se apresenta melhor, porém apresenta sinais de over-fitting em um das classes 
enquanto o SVM distribui um pouco melhor o scores entre as classes existentes. O adabooster não obteve um resultado satisfatório, e apresenta
um distribuição dos scores finais mais sparsas em relação as classes. Portanto, entre os classificadores o que se 
apresenta com melhor resultado é a SVM, devido a não existência de overfitting em relação as classes e ao mesmo tempo
apresentar uma acuráicia e scores médias melhores.